<a href="https://colab.research.google.com/github/lancejames221b/Blackjack-Tracker/blob/master/BlackJack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
from google.colab import drive
drive.mount('/content/drive')
!pip install silence_tensorflow
!pip install autokeras
import silence_tensorflow.auto
import tensorflow as tf
import autokeras as ak
import pandas as pd
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import time

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Wed Feb 17 16:18:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    42W / 300W |    489MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+------

In [52]:


def train(train_file_path, test_file_path, trials = None, epochs = None, dedup = False, sample = None):
    
    x_train = pd.read_csv(train_file_path)#.drop_duplicates()
    if duplicates and train_file_path is test_file_path: x_train = pd.read_csv(train_file_path).drop_duplicates()
    #x_train = x_train.astype(int)
    if sample: x_train = x_train.head(sample)
    print(len(x_train))

    x_train = x_train[['winloss', 'hand', 'dhand', 'lastwin','high', 'low', 'zeros']]
    y_train = x_train.pop('winloss')
    y_train = pd.DataFrame(y_train)
    x_test = pd.read_csv(test_file_path)#.drop_duplicates()
    if dedup: x_test = pd.read_csv(test_file_path).drop_duplicates()
    #x_test = x_test.astype(int)
    x_test = x_test[['winloss', 'hand', 'dhand', 'lastwin','high', 'low', 'zeros']]
    #x_test = x_test.sample(n=sys.argv[3])x_test = x_test[['winloss', 'lastwin']]
    print(len(x_test))
    y_test = x_test.pop('winloss')
    print("Loaded Training and Eval Dataset")
    clf = ak.StructuredDataClassifier(
    overwrite=True, max_trials = trials)
    # Feed the structured data classifier with training data.
    clf.fit(x_train, y_train, epochs = epochs)
    # Predict with the best model.
    predicted_y = clf.predict(x_test)
    # Evaluate the best model with testing data.
    print(clf.evaluate(x_test, y_test))
    # Feed the structured data classifier with training data.clf.fit(x_train, y_train)
    # Predict with the best model.predicted_y = clf.predict(x_test)
    # Evaluate the best model with testing data.print(clf.evaluate(x_test, y_test))

    model = clf.export_model()
    try:
        model.save("/content/drive/MyDrive/blackjack_model", save_format="tf")
    except:
        model.save("/content/drive/MyDrive/blackjack_model.h5")
    print("model exported")

def predict_test(csvfile, sample = None, dedup = False):
  from tensorflow.keras.models import load_model
  loaded_model = load_model("/content/drive/MyDrive/blackjack_model", custom_objects=ak.CUSTOM_OBJECTS, compile=True)
  x_test = pd.read_csv('hands.csv')#.drop_duplicates()
  if dedup: x_test = pd.read_csv('hands.csv').drop_duplicates()
  if sample: x_test = x_test.sample(sample)
  x_test = x_test[[ 'winloss', 'hand', 'dhand', 'lastwin','high', 'low', 'zeros']]
  y_test = x_test.pop('winloss')
  predictions = loaded_model.predict(x_test)
  predictions = np.argmax(predictions, axis=1)
  return zip(y_test, predictions)

In [56]:
train('hands.csv', 'training.csv', trials = 10, epochs = 500, duplicates = True)


900000
3378
Loaded Training and Eval Dataset

Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
structured_data...|True              |?                 
structured_data...|2                 |?                 
structured_data...|False             |?                 
structured_data...|0                 |?                 
structured_data...|32                |?                 
structured_data...|32                |?                 
classification_...|0                 |?                 
optimizer         |adam              |?                 
learning_rate     |0.001             |?                 

Epoch 1/500
22500/22500 [==============================] - 67s 3ms/step - loss: 0.8104 - accuracy: 0.6376 - val_loss: 0.7815 - val_accuracy: 0.6603
Epoch 2/500
22500/22500 [==============================] - 67s 3ms/step - loss: 0.7802 - accuracy: 0.6612 - val_loss: 0.7791 - val_accuracy: 0.6622
Epoch 3/500
22500/22500 [==============================] - 67

KeyboardInterrupt: ignored

In [41]:
x_test = pd.read_csv('hands.csv')#.drop_duplicates()

x_test = x_test.tail(10000)
x_test = x_test[[ 'winloss', 'hand', 'dhand', 'lastwin','high', 'low', 'zeros']]
y_test = x_test.pop('winloss')

In [42]:
from tensorflow.keras.models import load_model
loaded_model = load_model("/content/drive/MyDrive/blackjack_model", custom_objects=ak.CUSTOM_OBJECTS, compile=True)

In [43]:
predictions = loaded_model.predict(x_test)

In [44]:
predictions = np.argmax(predictions, axis=1)

In [45]:
testing = zip(y_test, predictions)

In [46]:
for i in testing:
  print(i)

Streaming output truncated to the last 5000 lines.
(0, 0)
(0, 0)
(1, 0)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(2, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(1, 0)
(0, 1)
(1, 0)
(0, 1)
(1, 1)
(1, 1)
(0, 1)
(0, 0)
(1, 1)
(0, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(2, 0)
(0, 0)
(0, 0)
(0, 0)
(2, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(1, 0)
(0, 0)
(0, 0)
(0, 0)
(1, 0)
(0, 0)
(2, 0)
(0, 0)
(1, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(1, 0)
(1, 0)
(2, 0)
(0, 0)
(1, 0)
(1, 0)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(0, 1)
(0, 0)
(2, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(1, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(1, 0)
(1, 1)
(1, 1)
(0, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(0, 1)
(0, 1)
(0, 1)
(1, 1)
(0, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
(2, 0)
(0, 0)
(1, 0)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(1, 1)
(0, 1)
(1, 1)
(1, 1)
(1, 1)
(0, 1)
(2, 0)
(1, 0)
(2, 0)
(0, 0)
(1, 0)
(0, 0)
(0, 

In [33]:
for i in testing:
  print(i)

('hand', 0)
('dhand', 0)
('lastwin', 0)
('high', 0)
('low', 0)
('zeros', 0)
